In [1]:
import pandas as pd
from IPython.display import display
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import os
print(f'diretório: \n{os.getcwd()}\n')

diretório: 
/home/cloud/Tensor GPU/P1 Cdados



In [3]:
#Le e separa as paginas da planilha, treinamento e teste
#sheet1, sheet2 = None, None
xls = pd.ExcelFile("Pokémon.xlsx")
df1 = pd.read_excel(xls, 0) #pagina 1 da planilha
df2 = pd.read_excel(xls, 1) #oagina 2 da polanilha
#print("Pagina 1 Treinamento:", df1.shape)
#print("Pagina 2 Teste:", df2.shape)

In [4]:
dados = df1
dados.head()

,Treinamento,relevante
0,besouros são nomeados em homenagem a pokémon h...,0
1,ash e goh ganham pokémon novos! uma nova rival...,1
2,alguém me dá a pelúcia do grookey pfv https://...,1
3,agora q eu e as minhas irmãs temos feito camin...,0
4,um pokémon apareceu. é um ninetales,0


In [5]:
dados = df1.rename(columns = {"Treinamento": "treinamento", "relevante": "nota"})

In [6]:
#Conta quantos caracteres cada tweet possui antes do filtro
dados['comprimento'] = [len(t) for t in dados.treinamento]
dados[dados.comprimento > 140].head()
contagem = dados
dados.head()

,treinamento,nota,comprimento
0,besouros são nomeados em homenagem a pokémon h...,0,68
1,ash e goh ganham pokémon novos! uma nova rival...,1,128
2,alguém me dá a pelúcia do grookey pfv https://...,1,61
3,agora q eu e as minhas irmãs temos feito camin...,0,148
4,um pokémon apareceu. é um ninetales,0,35


In [7]:
import string
import re
def filtro(tweet):
    tweet = ''.join(t for t in tweet if t not in string.punctuation)
    tweet = re.sub('((www\S+)|(http\S+))', 'urlsite', tweet)
    tweet = re.sub(r'\d+', 'contnum', tweet)
    tweet = re.sub(' +',' ', tweet)
    tweet = tweet.lower().strip()
    return tweet

#filtro de emojis
def filtro2(tweet):
    string_with_nonASCII = tweet
    encoded_string = string_with_nonASCII.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string


In [8]:
dados["treinamento"] = dados["treinamento"].apply(filtro)
dados["treinamento"] = dados["treinamento"].apply(filtro2)

In [9]:
#filtro de simbolos ou caracteries especiais que podem ter sobrado
simbolos = ["!",'"',"#","%","&","'","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","\\","]","^","_","`","{","|","}","~","–","http","https"]
for s in simbolos:
    dados['treinamento'] = dados['treinamento'].str.replace(s, ' ')
    dados['treinamento'] = dados['treinamento'].str.split().str.join(" ")



<ipython-input-9-b581da50640d>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dados['treinamento'] = dados['treinamento'].str.replace(s, ' ')


In [10]:
#Conta quantos caracteres cada tweet possui apos filtro
dados['novo_comprimento'] = [len(t) for t in dados.treinamento]
dados[dados.novo_comprimento > 140].head()
dados.head()

,treinamento,nota,comprimento,novo_comprimento
0,besouros so nomeados em homenagem a pokmon url...,0,68,50
1,ash e goh ganham pokmon novos uma nova rivalid...,1,128,107
2,algum me d a pelcia do grookey pfv urlsite,1,61,42
3,agora q eu e as minhas irms temos feito caminh...,0,148,143
4,um pokmon apareceu um ninetales,0,35,31


In [11]:
df = dados.loc[:,["treinamento", "nota"]]
df.head()

,treinamento,nota
0,besouros so nomeados em homenagem a pokmon url...,0
1,ash e goh ganham pokmon novos uma nova rivalid...,1
2,algum me d a pelcia do grookey pfv urlsite,1
3,agora q eu e as minhas irms temos feito caminh...,0
4,um pokmon apareceu um ninetales,0


In [12]:
df_relevante = df[df.nota == 1]
df_irrelevante = df[df.nota == 0]

In [13]:
relevante = df_relevante.treinamento.tolist()

palavra_relevante = []
for e in relevante:
    #tweet.append(filtro1(e).split())
    relevante = (e.split())
    for e in relevante:
            palavra_relevante.append(e)

serie_relevante = pd.Series(palavra_relevante)
serie_relevante_abasoluta = serie_relevante.value_counts()
serie_relevante_relativa = serie_relevante.value_counts(True)

In [14]:
irrelevante = df_irrelevante.treinamento.tolist()

palavra_irrelevante = []
for e in irrelevante:
    #tweet.append(filtro1(e).split())
    irrelevante = (e.split())
    for e in irrelevante:
            palavra_irrelevante.append(e)

serie_irrelevante = pd.Series(palavra_irrelevante)
serie_irrelevante_abasoluta = serie_irrelevante.value_counts()
serie_irrelevante_relativa = serie_irrelevante.value_counts(True)

In [15]:
total_relevante = serie_relevante_abasoluta.sum()
total_irrelevante = serie_irrelevante_abasoluta.sum()
total_palavras = serie_relevante_abasoluta.sum() + serie_irrelevante_abasoluta.sum()
total_palavras

5836

In [16]:
prob_relevante = total_relevante/total_palavras
prob_irrelevante = total_irrelevante/total_palavras
checagem = prob_relevante+prob_irrelevante
print("A probabilidade de ser relevante é: ", prob_relevante) # palavras
print("A probabilidade de ser irrelevante é: ", prob_irrelevante) # palavras
print("Checagem: ",checagem)

A probabilidade de ser relevante é:  0.2810143934201508
A probabilidade de ser irrelevante é:  0.7189856065798492
Checagem:  1.0


In [40]:
prob_rel = 1
prob_irr = 1

lista_tweets_relevantes = []
lista_tweets_irrelevantes = []




for tweet in df.treinamento:
    tweet = tweet.split()
    
    for e in range(len(tweet)):
        if tweet[e] in serie_relevante:
            prob_rel = (serie_relevante_abasoluta[tweet[e]])/(total_palavras)
            lista_tweets_relevantes.append(prob_rel)
        else:
            prob_rel = (0+1)/(total_relevante+total_palavras)
            lista_tweets_relevantes.append(prob_irr)
        
        if tweet[e] in serie_irrelevante:
            prob_irr = ((serie_irrelevante_abasoluta[tweet[e]])/(total_palavras))
            lista_tweets_irrelevantes.append(prob_irr)
        else:
            prob_irr = (0+1)/(total_irrelevante+total_palavras)
            lista_tweets_irrelevantes.append(prob_irr)
        
            #lista_tweets_relevantes.append(prob_relevante*prob_rel)
            #lista_tweets_irrelevantes.append(prob_irrelevante*prob_irr)
            
        
           
            

In [47]:
prob_irr

9.968102073365231e-05

In [48]:
prob_relevante

0.2810143934201508

In [ ]:
resultado = df

In [46]:
for e in lista_tweets_relevantes:
    for i in lista_tweets_irrelevantes:
        if e/i > i/e:
            resultado["Resultado"] = 1
            
        else:
            resultado["Resultado"] = 0
            

KeyboardInterrupt: 

### Conclusao

O classificador não funcionou como o esperado, pois mais de 90% dos itens analisados deram como irrelevante. Após Multiplas mudanças  e testes, fizemos algumas alterações pórem nesta última o classificador mostra sempre o mesmo resultado para relevantes e irrelevantes, sendo eles, 0.30 para relevantes e 9.96*10**-5 para irrelevantes.

O resultado esperado seria uma lista de valores para tweets relevantes e irrelevantes com valores diferentes para cada um deles. Após este resultado esperado, as listas seriam comparadas e o sesultado seria colocado no dataframe de treinamento.

Com isso, caso tivesse ocorrido sucesso nos testes, seria utilizado o dataframe de testes, e aplicado o mesmo codigo, assim poderiamos ter uma noção da porcentagem de acerto e porcentagem de erros do classificador.

Caso isso tivesse funcionado, poderia ser aplicado por exemplo em provas escritas de alunos. Cada um deles escreveria texto grande com as suas palavras e isso pode ser guardado. Caso aja uma supeita de cola ou plagio, os textos poderiam ser compardas e dependendo da porcentagem poderia ser um indicio de cola para ser investigado pela coordenação por exemplo. 